In [1]:
import pandas as pd
import refinitiv.data as rd
from datetime import datetime, timedelta
from pydash.arrays import chunk
from tqdm import tqdm
import numpy as np

rd.open_session(app_key= '') # insert api key here

<refinitiv.data.session.Definition object at 0x22d59b92900 {name='workspace'}>

In [2]:
def merge(year):
    
    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)

    cik_df = pd.read_parquet(f"Data/{year}/cik_{year}.parquet")
    ric_df = pd.read_parquet(f"Data/{year}/ric_{year}.parquet")
    
    # Convert the 'cik' column in both DataFrames to strings
    cik_df['cik'] = cik_df['cik'].astype(str)
    ric_df['cik'] = ric_df['cik'].astype(str).str.zfill(10)
    
    # Perform a right merge so that all rows from ric_df are kept
    merged_df = pd.merge(cik_df, ric_df, how='right', on='cik')
    merged_df['RIC'] = merged_df['RIC'].astype(str)
   
    return merged_df

In [4]:
Test = pd.read_parquet("Data/2016/Regression_Target_2016/2016_Regression_Stockprices.parquet")
Test

Date  Price Close          RIC
0      2016-01-29     6.384554  MGCD.OQ^A18
1      2016-02-01     6.272871  MGCD.OQ^A18
2      2016-02-02     6.319405  MGCD.OQ^A18
3      2016-02-03     6.282178  MGCD.OQ^A18
4      2016-02-04     6.282178  MGCD.OQ^A18
...           ...          ...          ...
132774 2017-02-23    18.073342      HNNA.OQ
132775 2017-02-24    17.833342      HNNA.OQ
132776 2017-02-27    18.066676      HNNA.OQ
132777 2017-02-28    17.793342      HNNA.OQ
132778 2017-03-01    18.613343      HNNA.OQ

[132779 rows x 3 columns]

In [3]:
def get_stockprices(ric_date_df, year):

    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)

    results = []  # To store individual DataFrames
        
    for _, row in tqdm(
        ric_date_df.iterrows(), 
        total=ric_date_df.shape[0],         # Explicit total for accurate progress tracking
        desc="Processing batches",          # Custom description for the progress bar
        unit="batch",                       # Custom unit name to display per iteration
        ncols=100,                          # Width of the progress bar (optional)
        mininterval=0.1): 


        extracted_date = row['full_date']
        extracted_ric = row['RIC']
        
        # First we construct the time interval
        date_object = extracted_date

        # Add 90 days
        date_object_plus_90 = date_object + timedelta(days=90)

        # Convert the new dates to an ISO8601 string
        event_date = date_object.strftime("%Y-%m-%d")
        date_plus_90 = date_object_plus_90.strftime("%Y-%m-%d")
        
        
        # Call get_history for the current batch        
        try:
            Stockprices_dataframe = rd.get_history(universe= extracted_ric, fields=[ "TR.PriceClose"], interval="1D",
               start = event_date, end = date_plus_90)         
            
            # Turn index 'date' into a column 
            Stockprices_dataframe_with_date = Stockprices_dataframe.reset_index()   
            
            # Add the RIC column to the resulting DataFrame
            Stockprices_dataframe_with_date['RIC'] = extracted_ric
            
            # Append DataFrame to results
            results.append(Stockprices_dataframe_with_date)  
            
        except Exception as e:
            print(f"Error processing batch {row}")

    
    for i, df in enumerate(results):
        with open(f'Data/{year}/Regression_Target_{year}/Price_Close_debug_{year}.txt', "a") as f:
            f.write(f"DataFrame {i} - dtype: {df['Price Close'].dtype}\n")
            f.write(f"{df['Price Close'].unique()}\n")
            f.write("===================================================================================\n")
        


    # Convert 'Price Close' to numeric in order to combat the following error:
    # ValueError: Error converting column "Price Close" to bytes using encoding None. Original error: could not convert string to float: ''
    for i in range(len(results)):
        results[i]['Price Close'] = pd.to_numeric(results[i]['Price Close'], errors='coerce')


    # Concatenate all results into a single DataFrame
    result = pd.concat(results, ignore_index=True)
    
        # Save the cleaned DataFrame
    result.to_parquet(f'Data/{year}/Regression_Target_{year}/{year}_Regression_Stockprices.parquet', engine='fastparquet')
    
    print(f"Regression Target Stockprices for {year} has been analyzed and saved into a parquet file.")


In [9]:
get_stockprices(merge("2016"), "2016")

Processing batches:  33%|████████████                         | 785/2398 [09:46<13:00,  2.07batch/s]C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1056:RuntimeWarning: invalid value encountered in cast
C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1080:RuntimeWarning: invalid value encountered in cast
Processing batches: 100%|████████████████████████████████████| 2398/2398 [28:35<00:00,  1.40batch/s]


Regression Target Stockprices for 2016 has been analyzed and saved into a parquet file.


In [10]:
get_stockprices(merge("2017"), "2017")

Processing batches: 100%|████████████████████████████████████| 2345/2345 [23:51<00:00,  1.64batch/s]


Regression Target Stockprices for 2017 has been analyzed and saved into a parquet file.


In [11]:
get_stockprices(merge("2018"), "2018")

Processing batches:  33%|████████████▏                        | 773/2335 [07:59<14:55,  1.74batch/s]C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1056:RuntimeWarning: invalid value encountered in cast
C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1080:RuntimeWarning: invalid value encountered in cast
Processing batches: 100%|████████████████████████████████████| 2335/2335 [22:42<00:00,  1.71batch/s]


Regression Target Stockprices for 2018 has been analyzed and saved into a parquet file.


In [12]:
get_stockprices(merge("2019"), "2019")

Processing batches: 100%|████████████████████████████████████| 2365/2365 [23:08<00:00,  1.70batch/s]


Regression Target Stockprices for 2019 has been analyzed and saved into a parquet file.


In [4]:
get_stockprices(merge("2020"), "2020")

Processing batches: 100%|████████████████████████████████████| 2409/2409 [26:20<00:00,  1.52batch/s]


Regression Target Stockprices for 2020 has been analyzed and saved into a parquet file.


In [5]:
get_stockprices(merge("2021"), "2021")

Processing batches: 100%|████████████████████████████████████| 2677/2677 [28:55<00:00,  1.54batch/s]


Regression Target Stockprices for 2021 has been analyzed and saved into a parquet file.


In [6]:
get_stockprices(merge("2022"), "2022")

Processing batches:  46%|████████████████▋                   | 1442/3105 [16:40<16:13,  1.71batch/s]

Error processing batch cik                   0001893219
full_date    2022-03-24 00:00:00
RIC                       ABP.OQ
RIC_clean                 ABP.OQ
Name: 1441, dtype: object


Processing batches: 100%|████████████████████████████████████| 3105/3105 [33:05<00:00,  1.56batch/s]


Regression Target Stockprices for 2022 has been analyzed and saved into a parquet file.


In [4]:
get_stockprices(merge("2023"), "2023")

Processing batches:  26%|█████████▌                           | 766/2952 [07:13<19:58,  1.82batch/s]

Error processing batch cik                   0001893219
full_date    2023-03-31 00:00:00
RIC                       ABP.OQ
RIC_clean                 ABP.OQ
Name: 765, dtype: object


Processing batches: 100%|████████████████████████████████████| 2952/2952 [27:45<00:00,  1.77batch/s]


Regression Target Stockprices for 2023 has been analyzed and saved into a parquet file.
